<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Для начала, импортируем и установим все библиотеки, которые будем использовать:

In [21]:
import pandas as pd
import numpy as np
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from nltk.tag import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split, cross_val_score
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

RANDOM_STATE = 42
TEST_SIZE = 0.25

Напишем удобную функцию для подгрузки данных, а затем подгрузим датасет:

In [13]:
def upload_function(data, sep, decimal):
    try:
        info = pd.read_csv(f'/datasets/{data}.csv', sep=sep, decimal=decimal)
    except: 
        info = pd.read_csv(f'https://code.s3.yandex.net/datasets/{data}.csv', sep=sep, decimal=decimal)
    return info

In [14]:
comments = upload_function('toxic_comments', ',', ',')
display(comments.sample(5))

,Unnamed: 0,text,toxic
27095,27132,PS: I also meant to say earlier that some of t...,0
142954,143108,"""\n\n Evacuation and flight to areas within Na...",0
128793,128925,Wrong country data on EUROPE PAGE \n\nI saw th...,0
36080,36122,"Wow man, IT's not original research. But I gue...",0
11074,11087,"""\n\nI do not """"hate"""" Columbia College; I hat...",0


In [15]:
comments = comments.drop('Unnamed: 0', axis=1)
display(comments.sample(5))

,text,toxic
72949,This petition is legitimate and belongs here t...,0
132298,The cases section(Which reads like a See also ...,0
114623,"""\n\n Sabra and Shatila - a genocide ? \n\nHi ...",0
21430,OI \n\nDO NOT PUT WARNINGS ON MY TALK PAGE! Yo...,0
30997,The entry before I modified it contained unsou...,0


In [16]:
comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [17]:
comments.duplicated().sum()

0

In [18]:
comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Пропусков и дубликатов в датасете нет, приступим к обработке текста. Нужно лемматизировать и векторизовать его. Напишем для этого функцию:

In [19]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
vectorizer = TfidfVectorizer(ngram_range=(1,2),
                            max_features=50000,
                            stop_words='english') 

def get_wordnet_pos(treebank_tag):
    # Конвертируем POS-тег NLTK в формат WordNet
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN


def transform_function(text):
    text = re.sub(r"[^a-zA-Z]", " ", text)
    tokens = nltk.word_tokenize(text.lower())
    pos_tags = pos_tag(tokens)
    # Лемматизация с учетом POS-тегов
    lemmatized = []
    for word, tag in pos_tags:
        if word not in stop_words and len(word) > 2 and word not in stop_words:
            pos = get_wordnet_pos(tag)
            lemma = lemmatizer.lemmatize(word, pos=pos)
            lemmatized.append(lemma)
    
    return " ".join(lemmatized)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [20]:
comments['clean_text'] = comments['text'].apply(transform_function)
comments

,text,toxic,clean_text
0,Explanation\nWhy the edits made under my usern...,0,explanation edits make username hardcore metal...
1,D'aww! He matches this background colour I'm s...,0,aww match background colour seemingly stick th...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man really try edit war guy constantly rem...
3,"""\nMore\nI can't make any real suggestions on ...",0,make real suggestion improvement wonder sectio...
4,"You, sir, are my hero. Any chance you remember...",0,sir hero chance remember page
...,...,...,...
159287,""":::::And for the second time of asking, when ...",0,second time ask view completely contradict cov...
159288,You should be ashamed of yourself \n\nThat is ...,0,ashamed horrible thing put talk page
159289,"Spitzer \n\nUmm, theres no actual article for ...",0,spitzer umm theres actual article prostitution...
159290,And it looks like it was actually you who put ...,0,look like actually put speedy first version de...


Разобьем датасет на выборки в соотношении 3:1:

In [22]:
X = comments['clean_text']
y = comments['toxic']

X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y,
                                                   test_size=TEST_SIZE,
                                                   random_state=RANDOM_STATE,
                                                   stratify=y)

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

Подготовка данных завершена, данные оказались чистыми, без пропусков и дубликатов. Текст был предобработан (лемматизирован и векторизован), датасет разбит на выборки. Можно приступать к машинному обучению.

## Обучение

Посмотрим несколько моделей и проверим их кросс-валидацией на тренировочной выборке:

In [23]:
log_model = LogisticRegression(class_weight='balanced')

cross_val_score(log_model, X_train, y_train, cv=5, scoring='f1', n_jobs=-1)

array([0.7484059 , 0.74732176, 0.74416038, 0.7418235 , 0.74261603])

In [24]:
tree_model = DecisionTreeClassifier(max_depth=10, min_samples_leaf=50, min_samples_split=20)

cross_val_score(tree_model, X_train, y_train, cv=5, scoring='f1', n_jobs=-1)

array([0.60571115, 0.59955996, 0.60131435, 0.60049356, 0.59308072])

In [25]:
lgb_model = lgb.LGBMClassifier(
    learning_rate=0.1,
    n_estimators=300,
    random_state=RANDOM_STATE,
    n_jobs=-1,
    verbose=1 
)

cross_val_score(lgb_model, X_train, y_train, cv=5, scoring='f1', n_jobs=-1)

[LightGBM] [Info] Number of positive: 9712, number of negative: 85863
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 36.673299 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 569427
[LightGBM] [Info] Number of data points in the train set: 95575, number of used features: 15909
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.101617 -> initscore=-2.179391
[LightGBM] [Info] Start training from score -2.179391
[LightGBM] [Info] Number of positive: 9712, number of negative: 85863
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 36.787023 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 568496
[LightGBM] [Info] Number of data points in the train set: 95575, number of used features: 15907
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.101617 -> initscore=-2.179391
[LightGBM] [Info] Start training from score -2.179

array([0.76876598, 0.75951762, 0.76396058, 0.77250409, 0.75669328])

In [26]:
np.array([0.76876598, 0.75951762, 0.76396058, 0.77250409, 0.75669328]).mean()

0.76428831

Лучшие результаты показала модель градиентного бустинга. Получим с ее помощью предсказания и проверим на тестовой выборке:

In [27]:
lgb_model.fit(X_train, y_train)
predictions = lgb_model.predict(X_test)

print('Метрика f1 модели на тестовой выборке:', f1_score(y_test, predictions))

[LightGBM] [Info] Number of positive: 12140, number of negative: 107329
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 61.276439 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 698605
[LightGBM] [Info] Number of data points in the train set: 119469, number of used features: 19573
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.101616 -> initscore=-2.179393
[LightGBM] [Info] Start training from score -2.179393
Метрика f1 модели на тестовой выборке: 0.7617980674975495


## Выводы

В ходе проекта решалась задача бинарной классификации по поиску токсичных комментариев для интернет-магазина "Викишоп". 

Для начала, данные были подгружены и предобработаны. Пропусков и дубликатов в датасете не оказалось, все предоставленные данные оказались достаточно чистыми. Затем, текст комментариев был лемматизирован и векторизован. Для лемматизации использовался *WordNetLemmatizer* и *POS-tagging*, для векторизации *TF-IDF-Vectorizer*. В датасете был создан новый столбец с лемматизированным текстом, затем заданы входной и целевой признаки, после чего было произведено разбиение на выборки. Все было готово к машинному обучению.

Лучшие результаты показала модель градиентного бустинга из библиотеки *Light-GBM*. Метрика *f1*, полученная при кросс-валидации на тренировочной выборке составила *0.76*, что полностью удовлетворяло условию поставленной задачи (не ниже 0.75). Финальным шагом проекта стало получение предсказаний модели на тестовой выборке, где метрика *f1* так же составила *0.76*.